# 准备

In [1]:
!pip install faiss-cpu
!pip install sentence-transformers
!pip install pandas

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

In [2]:
encoder = SentenceTransformer("/models/bge-large-zh-v1.5")

# 代码

In [4]:
%%time

data = [['六分仪（Sextant）是一种用于测量天体角度的仪器，常用于航海和天文导航。', '工具'],
['烤白薯是一道美味健康的小吃，下面是简单的制作步骤', '菜谱'],
['老鼠是一种常见的小型哺乳动物，属于鼠科。', '动物'],
['比瑞吉 优选系列 添加深海鱼油 成猫猫粮2kg', '电商'],
['奶酪是一种以牛、羊、山羊或其他动物的乳汁为原料，经过发酵和加工制作而成的食品。', '食品'],
['凯锐思 幼猫猫粮奶糕全价幼猫粮天然粮 【尝鲜装】幼猫粮500g','电商'],
['黑洞是宇宙中一种极为神秘的天体，它是一种极度密集的天体，其引力非常强大，甚至连光都无法逃离其吸引。','天体物理']]
df = pd.DataFrame(data, columns = ['文本', '分类'])

print('\n数据:')
display(df)

text = df['文本']
vectors = encoder.encode(text)

vector_dimension = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

search_text = '给小猫吃点啥?'
print('\n问题:', search_text)
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

k = index.ntotal
distances, ann = index.search(_vector, k=k)

results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})
merge=pd.merge(results,df,left_on='ann',right_index=True)

print('\n相似度排序')
display(merge)


数据:


,文本,分类
0,六分仪（Sextant）是一种用于测量天体角度的仪器，常用于航海和天文导航。,工具
1,烤白薯是一道美味健康的小吃，下面是简单的制作步骤,菜谱
2,老鼠是一种常见的小型哺乳动物，属于鼠科。,动物
3,比瑞吉 优选系列 添加深海鱼油 成猫猫粮2kg,电商
4,奶酪是一种以牛、羊、山羊或其他动物的乳汁为原料，经过发酵和加工制作而成的食品。,食品
5,凯锐思 幼猫猫粮奶糕全价幼猫粮天然粮 【尝鲜装】幼猫粮500g,电商
6,黑洞是宇宙中一种极为神秘的天体，它是一种极度密集的天体，其引力非常强大，甚至连光都无法逃离其吸引。,天体物理



问题: 给小猫吃点啥?

相似度排序


,distances,ann,文本,分类
0,1.146100,5,凯锐思 幼猫猫粮奶糕全价幼猫粮天然粮 【尝鲜装】幼猫粮500g,电商
1,1.357119,3,比瑞吉 优选系列 添加深海鱼油 成猫猫粮2kg,电商
2,1.376508,2,老鼠是一种常见的小型哺乳动物，属于鼠科。,动物
3,1.398010,1,烤白薯是一道美味健康的小吃，下面是简单的制作步骤,菜谱
4,1.458382,4,奶酪是一种以牛、羊、山羊或其他动物的乳汁为原料，经过发酵和加工制作而成的食品。,食品
5,1.755716,6,黑洞是宇宙中一种极为神秘的天体，它是一种极度密集的天体，其引力非常强大，甚至连光都无法逃离其吸引。,天体物理
6,1.862863,0,六分仪（Sextant）是一种用于测量天体角度的仪器，常用于航海和天文导航。,工具


CPU times: user 183 ms, sys: 31.7 ms, total: 215 ms
Wall time: 220 ms
